In [95]:
import pandas as pd
import numpy as np
import os
import json

In [96]:
# Load all available weather data into a dataframe
directory = "vc_api/weather-data/"
files = [file for file in os.listdir(directory) if not file.startswith(".")]

airport = []
date = []
precip = []
snow = []
snowdepth = []
windgust = []
cloudcover = []
icon = []

for file in files:
    with open(directory+file) as f:
        j = json.load(f)
    airport.append(file[:3]) # airport code
    date.append(j['days'][0]['datetime'])
    precip.append(j['days'][0]['precip'])
    snow.append(j['days'][0]['snow'])
#     snowdepth.append(j['days'][0]['snowdepth'])
    windgust.append(j['days'][0]['windgust'])
    cloudcover.append(j['days'][0]['cloudcover'])
#     icon.append(j['days'][0]['icon'])

# create DataFrame
d = {'origin': airport, 'fl_date': date, 'precip': precip, 'snow': snow,
     'windgust': windgust, 'cloudcover': cloudcover}
weather_df = pd.DataFrame(data=d).set_index(['fl_date', 'origin'])

In [109]:
# load your dataframe to add weather features to
df = pd.read_csv('', index_col=0)

In [113]:
# join weather columns by origin and destination
df = df.join(weather_df, on=['fl_date', 'origin'], how='left').rename(columns={'precip':'origin_precip',
                                                                              'snow':'origin_snow',
                                                                              'windgust':'origin_windgust',
                                                                              'cloudcover': 'origin_cloudcover'})
df = df.join(weather_df, on=['fl_date', 'dest'], how='left').rename(columns={'precip':'dest_precip',
                                                                              'snow':'dest_snow',
                                                                              'windgust':'dest_windgust',
                                                                              'cloudcover': 'dest_cloudcover'})

Engineer weather features

In [114]:
# Bin weather into levels
def precip_bins(val):
    if np.isnan(val):
        return np.nan
    elif val > 79:
        return 3
    elif val > 39:
        return 2
    elif val > 0:
        return 1
    else:
        return 0

def snow_bins(val):
    if np.isnan(val):
        return np.nan
    elif val > 1.8:
        return 2
    elif val > 0:
        return 1
    else:
        return 0
    
def wind_bins(val):
    if np.isnan(val):
        return np.nan
    elif val > 46:
        return 3
    elif val > 35:
        return 2
    elif val > 0:
        return 1
    else:
        return 0
    
def cloud_bins(val):
    if np.isnan(val):
        return np.nan
    elif val > 71:
        return 2
    elif val > 47:
        return 1
    else:
        return 0
    
df['origin_precip_cat'] = df['origin_precip'].apply(precip_bins)
df['origin_snow_cat'] = df['origin_snow'].apply(snow_bins)
df['origin_windgust_cat'] = df['origin_windgust'].apply(wind_bins)
df['origin_cloud_cat'] = df['origin_cloudcover'].apply(cloud_bins)
df['dest_precip_cat'] = df['dest_precip'].apply(precip_bins)
df['dest_snow_cat'] = df['dest_snow'].apply(snow_bins)
df['dest_windgust_cat'] = df['dest_windgust'].apply(wind_bins)
df['dest_cloud_cat'] = df['dest_cloudcover'].apply(cloud_bins)

In [115]:
df.head(5)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,...,dest_windgust,dest_cloudcover,origin_precip_cat,origin_snow_cat,origin_windgust_cat,origin_cloud_cat,dest_precip_cat,dest_snow_cat,dest_windgust_cat,dest_cloud_cat
1,2018-01-01,DL,3500,9E,N316PQ,3500,10397,ATL,"Atlanta, GA",10431,...,NaN,NaN,0,0,2.0,2,NaN,NaN,NaN,NaN
2,2018-01-01,DL,3513,9E,N316PQ,3513,10397,ATL,"Atlanta, GA",11995,...,NaN,NaN,0,0,2.0,2,NaN,NaN,NaN,NaN
5,2018-01-01,B6,18,B6,N236JB,18,12478,JFK,"New York, NY",10721,...,NaN,NaN,0,0,3.0,0,NaN,NaN,NaN,NaN
7,2018-01-01,B6,34,B6,N368JB,34,12478,JFK,"New York, NY",10785,...,NaN,NaN,0,0,3.0,0,NaN,NaN,NaN,NaN
11,2018-01-01,B6,89,B6,N985JB,89,12478,JFK,"New York, NY",14679,...,NaN,NaN,0,0,3.0,0,NaN,NaN,NaN,NaN


In [ ]:
# # Save file
# df.to_csv('')